<a href="https://colab.research.google.com/github/Bhandari007/Timeseries-and-sequence-models/blob/main/C4W3_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using a multi-layer LSTM for forecasting

# Using RNNs to predict time series

In this notebook, we'll be using Tensorflow's layers for processing sequence data such as Recurrent Layers or LSTMs to see how these approaches compare.